# Операции над объектами Pandas

На прошлом занятии мы начали изучать библиотеку Pandas и познакомились с ее основными структурами данных: `Index`, `Series` и `DataFrame`. Очевидно, что работа с данными не ограничивается исключительно их хранением и чтением, а зачастую включается в себя применение специальных операций и функций: логических, арифметических, математических, статистических и т.д. в зависимости от задачи. Сегодня мы с вами познакомимся с функциями и методами объектов Pandas, которые позволяют эффективно обрабатывать большие объемы различных данных.

**Необходимые импорты**:

In [ ]:
import timeit

import numpy as np
import pandas as pd
import seaborn as sns

## Векторизованные операции в стиле NumPy

Pandas во многом построена на массивах NumPy, а потому значительная часть векторизованных операций из NumPy может быть использована вместе с объектами `pd.Series` и `pd.DataFrame`. Однако, при использовании этих операций с объектами `Pandas` существуют свои тонкости. Рассмотрим эти тонкости детальнее.

### Сохранение индексов

Объекты `pd.Series` и `pd.DataFrame` - это не просто одномерные и двумерные массивы данных. Это структуры данных с явными типизированными индексами. Этот факт учитывается во время выполнения различных операций над объектами `pd.Series` и `pd.DataFrame`.  Так при выполнении унарных операций или при применении к этим объектам арифметических функций, Pandas будет использовать индексы исходного объекта в качестве индексов результата выполнения вычислений. Благодаря этому подходу разработчики получают легкий способ обновлять столбцы датафреймов, используя результаты выполнения различных операций.

Рассмотрим примеры сохранения индексов во время выполнения различных вычислений. 

In [ ]:
series = pd.Series(
    data=np.random.normal(size=5),
    index=list("ABCDE"),
)
series_exp = np.exp(series)

print(
    f"Original series:\n{series}",
    f"Series exp:\n{series_exp}",
    sep="\n\n",
)

Аналогичным образом сохранение индексов строк и индексов столбцов выполняется и для `pd.DataFrame`.

In [ ]:
row_amount, col_amount = 3, 4

data_frame = pd.DataFrame(
    data=np.random.normal(size=(row_amount, col_amount)),
    index=[f"row_{i + 1}" for i in range(row_amount)],
    columns=[f"col_{i + 1}" for i in range(col_amount)],
)
data_frame

In [ ]:
np.sin(data_frame * np.pi)

### Выравнивание индексов

Вторая, более важная особенность выполнения операций с объектами Pandas, заключается в выравнивании индексов. Из NumPy мы знаем, что при попытке выполнениях бинарных операций с одномерными массивами `np.ndarray`, мы получим ошибку. Поскольку массивы NumPy лежат в основе библиотеки Pandas, может создастся впечатление, что похожее поведение должно быть справедливо и для объектов `pd.Series` с разными индексами. Ведь `pd.Series` - это аналог одномерного массива `np.ndarray` с явно заданным индексом, а отличия в количестве элементов массивов `np.ndarray` аналогично отличиям в индексах объектов `pd.Series`.  Однако это ложное предположение. В Pandas возможно выполнение бинарных операций над объектами `pd.Series` с разными индексами. Более того, в результате этих операций получаются вполне предсказуемые и логичные результаты. Рассмотрим пример.

In [ ]:
populations = pd.Series(
    {
        "Moscow": 13149803,
        "Saint Petersburg": 5600044,
        "Novosibirsk": 1635338,
        "Ekaterinburg": 1539371,
    },
)
areas = pd.Series(
    {
        "Kazan": 515.8,
        "Ekaterinburg": 1112,
        "Moscow": 2511,
    },
)
population_density = populations / areas

print(f"Population density:\n{population_density}")

В данном примере мы определили `pd.Series` `populations`, в котором отражены численности населения российских городов. Также мы определили `pd.Series` `areas`, в котором отражены площади российских городов. Индексы определенных серий разные. Они имеют пересечения в виде ключей `"Ekaterinburg"` и `"Moscow"`, однако остальные ключи отличаются. Далее выполняется операция бинарного деления, чтобы определить плотность населения в российских городах. В результате выполнения мы получили объект `pd.Series`, в котором плотность населения определена только для тех значений индекса, которые присутствовали и в первой, и во второй серии. Остальным же значениям индекса соответствует странное значение `NaN`, о котором мы поговорим ниже. Однако, факт остается фактом - мы можем выполнять бинарные операции с объектами, обладающими разными индексами.

Рассмотрим детальнее результат выполнения такой операции. Во-первых, видно что индекс результирующей серии является объединением значений индексов операндов:

In [ ]:
is_index_united = np.all(
    population_density.index == populations.index.union(areas.index)
)

print(f"is index was united: {is_index_united}")

Во-вторых, значения индекса результата выполнения операции отсортированы в порядке возрастания. Поскольку в данном примере в качестве значений индекса были использованы названия городов, значения результирующего индекса отсортированы в алфавитном порядке:

In [ ]:
population_density.index

Аналогичные результаты будут справедливы и для объектов `pd.DataFrame`. Единственное отличие будет заключать в выравнивании индексов по двум измерениям.

In [ ]:
data_frame1 = pd.DataFrame(
    data=np.random.randint(0, 20, size=(2, 2)),
    columns=list("AB"),
)
data_frame1

In [ ]:
data_frame2 = pd.DataFrame(
    data=np.random.randint(0, 10, size=(3, 3)),
    columns=list("BAC"),
)
data_frame2

In [ ]:
data_frame1 + data_frame2

Единственный момент, который может смущать нас на данном этапе - наличие непонятного значения `NaN` в данных. При выполнении бинарных операций с объектами Pandas, мы можем предотвратить появление этого значения в результате. Чтобы это сделать, нам придется воспользоваться другой формой бинарных операций - бинарными операциями в форме методов объектов `pd.Series` и `pd.DataFrame`. При использовании данной формы бинарных операций с помощью аргумента `fill_value` мы можем явно указать, какое значение стоит использовать в тех случаях, когда ключ отсутствует в одном из операндов. В этом случае результат будет выглядеть так:

In [ ]:
data_frame1.add(data_frame2, fill_value=0)

Таблица соответствия методов и бинарных операций:

| Оператор Python | Метод объекта Pandas |
|---|---|
| + | add() |
| - | sub(), subtract() |
| * | mul(), multiply() |
| / | truediv(), div(), divide() |
| // | floordiv() |
| % | mod() |
| ** | pow() |

### Транслирование (Broadcasting)

Во всех предыдущих примерах для выполнения бинарных операций мы использовали операнды одних и тех же типов данных. Однако мы можем выполнять бинарные операции с операндами различных типов данных. Например, мы можем вычитать объект типа `pd.Series` из объекта типа `pd.DataFrame`. В случае, если индексы объектов совпадают, результат будет аналогичен вычитанию одномерного массива `np.ndarray` из двумерного массива `np.ndarray`. Т.е. будет происходить транслирование одномерного объекта по уже знакомым нам правилам:

In [ ]:
row_amount, col_amount = 3, 4

data_frame = pd.DataFrame(
    data=np.random.randint(0, 10, size=(row_amount, col_amount)),
    index=[f"row_{i + 1}" for i in range(row_amount)],
    columns=[f"col_{i + 1}" for i in range(col_amount)],
)
data_frame

In [ ]:
data_frame - data_frame.loc["row_2"]

По умолчанию вычитание происходит построчно. Однако далеко не всегда мы хотим выполнять операции построчно. Существуют случае, когда нам необходимо выполнить некоторую операцию по столбцам. В этом случае нам придется воспользоваться методами объектов Pandas для выполнения бинарных операций, а также указать значение аргумента `axis=0`.

In [ ]:
data_frame.subtract(data_frame["col_1"], axis=0)

Также далеко не всегда индексы операндов могут совпадать. В этом случае будет происходить уже знакомое нам выравнивание.

In [ ]:
first_row_odd_columns = data_frame.iloc[0, ::2]

print(
    "even columns values from first row:\n"
    f"{first_row_odd_columns}"
)

In [ ]:
data_frame - first_row_odd_columns

## NaN. Обработка отсутствующих данных

### Что такое NaN и откуда он взялся?

В предыдущих примерах мы столкнулись со значением `NaN` в наших данных. Это специальное значение, которое Pandas использует для того, чтобы помечать отсутствующие данные. `NaN` - акроним, составленный из первых букв фразы *not a number*. Давайте разберемся, как еще `NaN` может попасть в наши данные, как задать это значение самостоятельно, и как с ним работать.

Часто при работе с реальными данными, нам приходится сталкиваться с неполными данными. Например, в результате некоторого статистического опроса несколько респондентов забыли заполнить графу возраста. В этом случае некоторые данные о респондентах будут неполные. Неполные данные легко представить на бумаге, но как представлять неполные данные в коде?  В Python для этих целей мы использовали объект-синглтон None. Однако при работе с NumPy использование None приводит к печальным последствиям. Рассмотрим пример.

In [ ]:
array = np.array([1, 2, None, 4])

print(
    f"array data:\n{array}",
    f"array dtype: {array.dtype}",
    sep="\n\n",
)

Из этого кода следует следующее. Если NumPy встречает в данных объект `None`, он осуществляет повышающее преобразование типов. После преобразования типов в массиве `np.ndarray` будут лежать данные типа `object`. Т.е. NumPy будет воспринимать содержимое  массива, как обычные объекты Python. Это значит, что работа с таким массивом не будет отличаться от работы с обычными списками Python. Никакая векторизация в таком случае невозможна.

In [ ]:
setup = "import numpy as np"
template = "np.arange(int(1e6), dtype=%s).sum()"
iteration_amount = 1000

for dtype in ["np.object_", "np.int32"]:
    print(f"dtype: {dtype}")
    time_per_iter = timeit.timeit(
        setup=setup,
        stmt=template % dtype,
        number=iteration_amount,
    ) / iteration_amount
    print(f"time_taken: {time_per_iter:.4f}s;", end="\n\n")

Более того, мы даже не сможем выполнять некоторые агрегирующие операции над содержимым такого массива. Из-за наличия в данных `None` мы с большой долей вероятности получим ошибку.

In [ ]:
print(f"array:\n{array}")
sum_of_elements = array.sum()

Чтобы обойти все эти ограничения, в NumPy реализовано специальное сигнальное значение `np.nan` в соответствии со стандартом IEEE. Фактически `np.nan` - это специальное число с плавающей точкой, используемое в качестве признака отсутствия данных. 

In [ ]:
array = np.array([1, 2, np.nan, 3])

print(
    f"array data:\n{array}",
    f"array dtype: {array.dtype}",
    sep="\n\n",
)

По своей природе значение `np.nan` похоже на вирус, который заражает собой любые данные, т.к. при выполнении любой операции с использованием `np.nan` результат также будет представлять собой `np.nan`.

In [ ]:
print(
    f"addition: {np.nan + 42}",
    f"multiplication: {np.nan * 42}",
    f"agregation: {array.sum()}",
    f"safe agregation: {np.nansum(array)}",
    sep="\n",
)

Поскольку в основе Pandas лежит NumPy, мы можем использовать значение `np.nan`, чтобы помечать отсутствующие данные. Также разработчики Pandas добавили возможность использования `None` для того, чтобы помечать отсутствующие данные. Таким образом мы имеем два взаимозаменяемых способа задания пропущенных значений.

In [ ]:
series = pd.Series([1, None, 2, np.nan])
print(f"series:\n{series}")

Также обращаем ваше внимание, что при появлении в данных `NaN`, Pandas может произвести повышающее преобразование типов. Так, в данном примеры изначальный тип данных объекта `pd.Series` был `int32`, т.е. тип данных был целочисленный. После того, как мы пометили данные под индексом 0 как отсутствующие, Pandas произвел повышающее преобразование типов до `float64`. Это произошло, потому что `np.nan` - это число с плавающей точкой, а объект `None`, используемый для пометки отсутствующих данных, Pandas неявно преобразует в `np.nan`.

In [ ]:
series = pd.Series(np.arange(4))
print(f"original series:\n{series}", end="\n\n")

series[0] = None
print(f"corrupted series:\n{series}")

### Выявление пустых значений

Первое, что стоит сделать с полученными данными - определить, присутствуют ли в них пропуски или нет. И если пропуски присутствуют, желательно понимать, где именно. Определить положения `NaN` в данных можно с помощью метода `isnull`.

In [ ]:
series = pd.Series([1, None, 2, 3, np.nan])
print(f"series:\n{series}")

In [ ]:
mask_data_missed = series.isnull()
print(f"missed data mask:\n{mask_data_missed}")

В Pandas также определен антипод метода `isnull` - `notnull`, который позволяет определить булеву маски для данных, не являющихся `NaN`.

In [ ]:
print(
    f"corrupted data:\n{series[mask_data_missed]}",
    f"correct data:\n{series[series.notnull()]}",
    sep="\n\n",
)

### Удаление пустых значений

Установив наличие `NaN` в данных, необходимо решить, что с ними делать. Редко в каких задачах уместно оставлять пропущенные данные. Обычно от пропусков или избавляются, или пытаются их заполнить по определенным правилам. Если данных очень много, пропусков очень мало, а их заполнение нецелесообразно, может потребоваться простое удаление таких данных из Pandas объектов. Это можно сделать с помощью метода `dropna`. 

In [ ]:
series = pd.Series([1, None, 2, 3, np.nan])
print(f"series:\n{series}")

In [ ]:
print(
    f"correct data:\n{series.dropna()}",
)

В случае работы с объектом `pd.Series` удаление `NaN` довольно прямолинейно. Однако при удалении данных из `pd.DataFrame` возникают сложности. Дело в том, что мы не можем удалять из датафрейма отдельные ячейки с данными. Можно удалить только строку или столбец целиком. По умолчанию `dropna` удаляет строки, содержащие хотя бы одно значение `NaN`.

In [ ]:
data_frame = pd.DataFrame(
    data=[
        [1, 2, np.nan],
        [4, 5, 6],
        [np.nan, 8, np.nan],
    ],
    columns=list("ABC")
)
data_frame

In [ ]:
data_frame.dropna()

Используя аргумент `axis`, мы можем указать измерение, вдоль которого должно анализироваться наличие пропусков. В примере ниже будут удалены все столбцы, содержащие хотя бы одно значение `NaN`.

In [ ]:
data_frame.dropna(axis="columns")

Так же функция `dropna` позволяет настраивать стратегии удаления строк и столбцов из датафрейма. Как говорилось выше, по умолчанию для удаления строки или столбца достаточно наличия хотя бы одного значения `NaN`. Однако такая стратегия может быть не всегда уместной. Часто в нашем распоряжении не так много данных, чтобы мы могли позволить себе выбрасывать строки или столбцы только из-за наличия одного `NaN`. Именно поэтому Pandas позволяет настроить правила, в соответствии с которым будет происходить удаление строк или столбцов. Так мы можем установить минимальное число значений отличных от `NaN`, необходимое для сохранения строки или столбца в датафрейме. Сделать это можно с помощью параметра `thresh`.

In [ ]:
data_frame.dropna(thresh=2)

Также при необходимости мы можем удалять только те строки или столбцы, которые полностью заполнены `NaN`.

In [ ]:
data_frame.iloc[-1, 1] = np.nan
data_frame

In [ ]:
data_frame.dropna(how="all")

### Заполнение пустых значений

Выше упоминалось, что в нашем распоряжении обычно не так много данных, чтобы мы могли позволить себе их сокращение путем удаления. Именно поэтому часто более оптимальной стратегией для обработки пропусков является заполнение, а не удаление. Заполнить пропущенные данные в Pandas можно с помощью методов `fillna`, `ffill` и `bfill`.

С помощью метода `fillna` можно заполнить недостающие данные переданным значением.

In [ ]:
series = pd.Series([None, 1, 2, 3, np.nan])
print(
    f"series original:\n{series}",
    f"series filled:\n{series.fillna(0)}",
    sep="\n\n",
)

`ffill` и `bfill` используют, соответственно, предшествующее и следующее значение для заполнения пропусков. 

In [ ]:
print(
    f"series original:\n{series}",
    f"series forward fill:\n{series.ffill()}",
    f"series backward fill:\n{series.bfill()}",
    sep="\n\n",
)

В случае с `DataFrame` справедливо все, сказанное выше.

In [ ]:
data_frame = pd.DataFrame(
    [
        [1, 2, np.nan],
        [4, 5, 6],
        [np.nan, 9, np.nan],
    ],
    columns=list("ABC"),
)
data_frame

In [ ]:
data_frame.fillna(0)

Мы также можем использовать объект `pd.Series`, чтобы заполнять пропущенные значения в разных столбцах по-разному.

In [ ]:
fill_values = pd.Series(
    data=[42, 69],
    index=list("AC"),
)
data_frame.fillna(fill_values)

Из-за двумерной природы объекта `pd.DataFrame` при использовании методов `ffill` и `bfill` мы можем выбирать размерность, вдоль которой будут использоваться предшествующие и следующие значения для заполнения пропусков. По умолчанию пропуски заполняются вдоль столбцов.

In [ ]:
data_frame.ffill()

In [ ]:
data_frame.ffill(axis="columns")

## Операции агрегирования

Объекты Pandas также поддерживают операции агрегирования. Поскольку простые операции агрегирования ничем не отличаются от соответствующих операций в NumPy, не будет заострять на них наше внимание.

In [ ]:
planets_data = sns.load_dataset("planets")
planets_data.sample(n=5)

In [ ]:
print(
    f"mean orbital period: {planets_data['orbital_period'].mean():.2f};",
    f"first year of research: {planets_data['year'].min()};",
    f"last year of research: {planets_data['year'].max()};",
    sep="\n",
)

С помощью метода `unique` возможно получение уникальных значений, хранящихся в объекте типа `pd.Series`. 

In [ ]:
planets_data["method"].unique()

Также в Pandas реализован метод `describe`, который позволяет вычислить основные статистики числовых данных. Эта функция может быть полезна при проведении разведывательного анализа данных.

In [ ]:
planets_data.dropna().describe()

## Операции со строковыми данными

Обсуждая NumPy, мы с вами занимались исключительно работой с числовыми данными. Теперь, работая с таблицами Pandas, нам нередко будут встречаться строковые данные. Необходимо уметь эффективно обрабатывать строковые данные и уметь пользоваться векторизованными строковыми операциями. Это можно сделать, используя специальный атрибут объектов `pd.Series` и `pd.DataFrame` - `str`.

In [ ]:
names = pd.Series(
    data=["john", "Paul", "george", "RINGO"],
)

print(
    f"names:\n{names}",
    f"names corrected:\n{names.str.capitalize()}",
    sep="\n\n",
)

В Pandas реализовано множество векторизованных операций для работы со строками, являющееся аналогом множества методов строк в Python. Если вы знакомы с оригинальными методами, вы без труда сможете разобраться с логикой работы аналогов в Pandas. Ниже приведем пару примеров.

In [ ]:
names = pd.Series(
    data=[
        "John Lennon",
        "Paul MacCartney",
        "George Harrison",
        "Ringo Starr",
    ],
)

print(
    f"names lens:\n{names.str.len()}",
    f"correction mask:\n{names.str.istitle()}",
    f"names split:\n{names.str.split()}",
    sep="\n\n",
)

Также в Pandas реализованы векторизованные операции для работы с регулярными выражениями.

In [ ]:
print(
    f"pattern matching:\n{names.str.match(r'[A-Za-z]+')}",
    f"pattern inclusion:\n{names.str.contains(r'[Jj]ohn')}",
    f"pattern finding:\n{names.str.findall(r'^[^AEIOUY]*[^aeiouy]$')}",
    sep="\n\n",
)

Также реализованы операции векторизованного среза строк.

In [ ]:
print(
    f"explicit slice:\n{names.str.slice(0, 3)}",
    f"implicit slice:\n{names.str[:3]}",
    sep="\n\n",
)

## Задача 1. My heart will go on

Датасет **titanic** из библиотеки `Seaborn` содержит информацию о пассажирах легендарного корабля Титаник, который затонул в 1912 году после столкновения с айсбергом. Этот набор данных часто используется для обучения и тестирования алгоритмов машинного обучения, особенно в задачах бинарной классификации (выжил / не выжил).

**Описание данных**

| Поле         | Тип      | Описание |
|--------------|----------|----------|
| `survived`   | int      | Выжил (1) или не выжил (0) |
| `pclass`     | int      | Класс билета (1, 2, 3) |
| `sex`        | str      | Пол (`male`/`female`) |
| `age`        | float    | Возраст |
| `sibsp`      | int      | Количество братьев/сестёр/супругов на борту |
| `parch`      | int      | Количество родителей/детей на борту |
| `fare`       | float    | Стоимость билета |
| `embarked`   | str      | Порт посадки (`C`=Cherbourg, `Q`=Queenstown, `S`=Southampton) |
| `class`      | str      | Класс билета (`First`, `Second`, `Third`) |
| `who`        | str      | Категория: `man`, `woman` или `child` |
| `adult_male` | bool     | Является ли взрослым мужчиной |
| `deck`       | str      | Палуба |
| `embark_town`| str      | Название порта посадки |
| `alive`      | str      | Выжил (`yes`/`no`) |
| `alone`      | bool     | Путешествовал один |

**Загрузка датасета**

In [ ]:
titanic_data = sns.load_dataset("titanic")
titanic_data.sample(5)

**Задача**

Ниже описаны 10 небольших заданий, которые вам необходимо решить.

**Подсказка**:

В некоторых заданиях вам может быть полезен метод `value_counts`.

### Часть 1

Определите число пропущенных данных для каждого столбца таблицы `titanic_data`.

In [ ]:
# ваш код

### Часть 2

Удалите все столбцы, количество пропусков в которых превышает половину количества строк в таблице.

После того, как вы удалите все столбцы, нарушающие описанное условие, удалите все строки, количество элементов в которых превышает половину количества столбцов.

In [ ]:
# ваш код

### Часть 3

Если вы сделали все правильно, больше всего пропусков должно остаться в столбце `"age"` - 177. Их необходимо заполнить. Заполним пропуски следующим образом:
- Если значение столбца `"who"="man"`, пропуск необходимо заполнить медианным значением известных возрастов мужчин, округленным до ближайшего целого числа;
- Если значение столбца `"who"="woman"`, пропуск необходимо заполнить медианным значением известных возрастов женщин, округленным до ближайшего целого числа;
- Если значение столбца `"who"="child"`, пропуск необходимо заполнить медианным значением известных возрастов детей, округленным до ближайшего целого числа;

In [ ]:
# ваш код

### Часть 4

Удалите все строки, в которых осталось больше одного пропуска. Если вы все сделали правильно, после этого действия в таблице не должно остаться пропусков.

In [ ]:
# ваш код

### Часть 5

Определите название города, из которого отправилось больше всего пассажиров.

In [ ]:
# ваш код

### Часть 6

Определите процент выживших пассажиров от числа пассажиров, оставшихся в таблице после очистки данных. Ответ округлите до 2 знаков после запятой.

In [ ]:
# ваш код

### Часть 7

Определите число выживших пассажиров для каждого пункта отправления. В ответе должен получиться объект типа `pd.Series`, индексы которого - названия пунктов отправления, а значения - число выживших пассажиров.

In [ ]:
# ваш код

### Часть 8

Определите процент выживших пассажиров в каждом классе. Значения округлите до 2 знаков после запятой. В ответе должен получиться объект типа `pd.Series`, индексы которого - названия классов, а значения - процент выживших пассажиров.

In [ ]:
# ваш код

### Часть 9

Будем считать, что пассажиры, купившие билет **НЕ МЕНЕЕ** чем за $100, считаются богатыми. Определите процент выживших среди богатых пассажиров. Ответ округлите до 2 знаков после запятой. В ответе должно получиться число. 

In [ ]:
# ваш код

### Часть 10

Определите количество детей, путешествовавших в одиночку.

In [ ]:
# ваш код

Какие выводы вы можете сделать о выживших пассажирах Титаника? 